In [9]:
import requests
class OntoTransServer(object):
    """ The OntoTransServer object represent a remote OntoTrans REST API on the network. """
    
    def __init__(self, url):
        self.url = url
        
    def create_dataresource(self, uri, mediatype):
        return DataResource()
    
    def say_hello(self):
        
        """ Say hello """
        response = requests.get(f'{self.url}/test')
        if response.status_code != 200:
            raise ApiError(f'Cannot say hello: {response.status_code}')
        return response.text
        
server = OntoTransServer('http://ontoapi:8000')
server.say_hello()

'"hello from Lian"'

In [39]:
from abc import ABC, abstractmethod
import json

ONTOSERVER = 'http://ontoapi:8000'

class Pipe(object):
    """ Pipe object in a pipe-and-filter pattern """
    def __init__(self, i):
        self.input = i
        
    def get(self):
        return self.input.get()
        
        
class DownloadStrategy(ABC):
    """
    """
    @abstractmethod
    def read(self, uri: str):
        pass


class Filter(ABC):
    """ Filter operation to be executed in a pipeline """
    def __init__(self, msg):
        self.input_pipe = None
        self.msg = msg
        self.server = OntoTransServer(ONTOSERVER)
    
    def _set_input(self, input_pipe : Pipe):
        self.input_pipe = input_pipe
        
    def __rshift__(self, other):  # A (this) >> B (other)
        
        # construct a pipe, and set the filter_url in the pipe
        p = Pipe(self)
        
        # set the input pipe on the right-hand side filter
        # post.(other_url, data{input_pipeid: p.pipe-Id})
        other._set_input(p)
        return other
    
    @abstractmethod
    def get(self):        
        pass
    
class DataResource(Filter):
    """ Context class for the DataResource Strategy Interfaces for managing i/o operations """
    
    def __init__(self, url : str, media_type : str = None):
        Filter.__init__(self, url)
        self.url = url
        print (f'{ONTOSERVER}/dataresource')
                
        data = {         
         'downloadUrl': url, 
         'mediaType': media_type}
        response = requests.post(f'{ONTOSERVER}/dataresource', 
                                 data = json.dumps(data))        
        r = response.json()
        print (r['resource_id'])
        self.dataresource_id = r['resource_id']
        
        
    def get(self):
        """get""" 
        
        response = requests.get(f'{ONTOSERVER}/dataresource/{self.dataresource_id}/read')
        if response.status_code != 200:
            raise ApiError(f'Cannot read from dataresource: {response.status_code}')
        return response.content        
    
    def datamodel(self):
        """generate datamodel """
        response = requests.get(f'{ONTOSERVER}/dataresource/{self.dataresource_id}/datamodel')
        if response.status_code != 200:
            raise ApiError(f'Cannot read from dataresource: {response.status_code}')
        return response.content        

class DataSource(Filter):
    def get(self):        
        if self.input_pipe:
            print ("->", self.input_pipe.get())
        return (self.server.say_hello())
        
        
class DataMapper(Filter):
    def get(self):
        if self.input_pipe:
            print ("mapping", self.input_pipe.get())
        return "mapped"
    


http://ontoapi:8000/dataresource
20da2242-fcce-49f7-953e-4feb698137ca


{'uri': 'com.sintef.soft/ontology/v1/generated_from_csv20307235_b9f9_4948_82bb_681badf5dbe6',
 'dimensions': {'nrows': 'Number of elements'},
 'properties': {'Index': {'type': 'int64',
   'label': 'Index',
   'description': '',
   'unit': '',
   'shape': ['nrows']},
  ' "Mass (kg)"': {'type': 'float64',
   'label': ' "Mass (kg)"',
   'description': '',
   'unit': '',
   'shape': ['nrows']},
  ' "Spring 1 (m)"': {'type': 'float64',
   'label': ' "Spring 1 (m)"',
   'description': '',
   'unit': '',
   'shape': ['nrows']},
  ' "Spring 2 (m)"': {'type': 'float64',
   'label': ' "Spring 2 (m)"',
   'description': '',
   'unit': '',
   'shape': ['nrows']}}}

In [ ]:
ds = DataResource('https://people.sc.fsu.edu/~jburkardt/data/csv/hooke.csv', 'text/csv')
model = json.loads(ds.datamodel())


In [45]:
from soft.rdf import Turtle

t = Turtle.dumps(model)
print (t)



@prefix : <http://example.com/entity#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix soft: <http://www.quaat.com/ontologies#> .
@base <http://example.com/entity> .



:Entity_02072554_4c24_4235_982d_d99d17c639a7 rdf:type owl:NamedIndividual ,
                                             soft:Entity ;
                                             soft:uri "com.sintef.soft/ontology/v1/generated_from_csv6d029b52_20d8_4357_9388_5e9cba748c60"^^xsd:anyURI ;
                                             rdfs:label "Entity"@en .


#################################################################
#    Dimensions
#################################################################



:Dimension_7e69ab3a_2381_44ff_8444_b33078aa3d39 rdf:type owl:NamedIndividual ,
        